##### 前面介绍过用于分类的线性模型，其中为了将原来在实数域上的结果映射到（0，1）区间，引入了逻辑斯蒂函数。
##### 而接下来的线性回归问题中，预测目标直接是实数域上面的数值，因此优化目标变得简单，即最小化预测结果和真实值之间的差异
##### argminL(w,b) = argmin sum(f(w,x,b)-y^i)2
##### 同样是需要计算w和b，有两种方法，一种是通过精确计算，另外一种是比较快速的随机梯度下降算法。Stochastic Gradient Descend

###### 使用波士顿房价数据来学习线性回归

### 1.导入数据

In [1]:
from sklearn.datasets import load_boston
import numpy as np
df = load_boston()
print(df.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

### 2.数据分割

In [2]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.data,df.target,test_size=0.25,random_state=123)

D:\softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
x_train.shape

(379, 13)

In [4]:
x_test.shape

(127, 13)

In [5]:
#对目标值进行分析
print("最大值：%s,最小值：%s,离差：%s,均值：%s"%(np.max(df.target),np.min(df.target),np.max(df.target)-np.min(df.target),np.mean(df.target)))

最大值：50.0,最小值：5.0,离差：45.0,均值：22.5328063241


In [7]:
#从上面分析的结果来看，差异较大，为了避免个别较大的属性参数对整个线性回归模型产生的主导现象，这里需要对训练数据和目标数据都要标准化处理

### 3.测试数据与训练数据标准化处理

In [6]:
from sklearn.preprocessing import StandardScaler
#特征标准化器
ss_X = StandardScaler()
#目标标准化器
ss_Y = StandardScaler()

#标准化
x_train = ss_X.fit_transform(x_train)
x_test = ss_X.transform(x_test)

y_train = ss_Y.fit_transform(y_train.reshape(-1,1))
y_test = ss_Y.transform(y_test.reshape(-1,1))

In [7]:
x_train.shape

(379, 13)

In [8]:
x_test.shape

(127, 13)

### 4.使用线性回归和SGDRegressor对波士顿房价做预测

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

lr = LinearRegression()
lr.fit(x_train,y_train)
lr_predict = lr.predict(x_test)

sgdr = SGDRegressor()
sgdr.fit(x_train,y_train.ravel())
sgdr_predict = sgdr.predict(x_test)

D:\softwares\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### 5.性能评估

#### 回归算法的性能评估和分类算法不一样，回归算法的预测值比一定精确到具体的目标值，但是我们可以通过计算预测值与目标值之间的距离来度量性能。
#### 常用的度量指标是平均绝对误差Mean Absolute Error MAE和均方误差 Mean Squared Error MSE来度量，但是单单使用这两个指标还不够，
#### 因为在不同的回归模型中，数据稍有变化MAE和MSE都会发生很大的变化，欠缺不同问题的可比性，因此我们要使用一个具有统计意义的指标，如同
#### 分类任务中的精确率、准确率等。在回归中使用R-squared指标来衡量回归结果的波动可被真实值验证的百分比，他在一定程度上可以提现模型在数值
#### 回归方面的能力
#### R-squared计算方法
#### R^2 = 1 - (SSres/SStot)
#### SSres = sum((yi-f(xi))^2)
#### SStot = sum((yi-ymean)^2)
#### MSE = SStot / m
#### MAE = SSabs/m
#### SSabs = sum((abs(yi-ymean)))

In [10]:
print('linearRegression R值：',lr.score(x_test,y_test))
print('sgdregressor R    值：',sgdr.score(x_test,y_test))

linearRegression R值： 0.686435486389
sgdregressor R    值： 0.665446659784


#### 6.使用scikit-learn内置的评价函数进行性能评估

In [12]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [13]:
print('linearRegression R-score: %s MAE: %s MSE: %s'%(r2_score(y_test,lr_predict),
                                                  mean_absolute_error(y_test,lr_predict),mean_squared_error(y_test,lr_predict)))

print('sgdregressor R-score: %s MAE: %s MSE: %s'%(r2_score(y_test,sgdr_predict),
                                                  mean_absolute_error(y_test,sgdr_predict),mean_squared_error(y_test,sgdr_predict)))

linearRegression R-score: 0.686435486389 MAE: 0.370057351171 MSE: 0.28706703603
sgdregressor R-score: 0.665446659784 MAE: 0.371131684462 MSE: 0.306282221363


In [30]:
#使用StandarScalar的inverse_transfor得到标准化值得原始值
print('linearRegression MAE:%s,MSE:%s'%(mean_absolute_error(ss_Y.inverse_transform(lr_predict),ss_Y.inverse_transform(y_test)),
                                       mean_squared_error(ss_Y.inverse_transform(lr_predict),ss_Y.inverse_transform(y_test))))

print('sgdregressor MAE:%s,MSE:%s'%(mean_absolute_error(ss_Y.inverse_transform(sgdr_predict),ss_Y.inverse_transform(y_test)),
                                       mean_squared_error(ss_Y.inverse_transform(sgdr_predict),ss_Y.inverse_transform(y_test))))

linearRegression MAE:3.43633538867,MSE:24.753484526
sgdregressor MAE:3.46558777432,MSE:26.4424961259


#### 7.特点分析

#### 从结果可以看到线性回归的性能比随机梯度下降的性能要好，但是对于大数据集，随机梯度下降可以在不损失过多精度的情况下快速的完成预测。sklearn官网
#### 推荐数据量大于10万的数据的回归操作，采用SGDRegressor